In [17]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import geocoder

In [18]:
wiki_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
resp = requests.get(wiki_url).text

In [19]:
soup = BeautifulSoup(resp, 'xml')#Beautiful Soup to Parse the url page


In [20]:
table=soup.find('table')


In [21]:
column_names=['Postalcode','Borough','Neighbourhood']
df = pd.DataFrame(columns=column_names)

In [22]:
# extracting information from the table
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [23]:
df.head()


,Postalcode,Borough,Neighbourhood


In [24]:
# remove rows where Borough is 'Not assigned'
df=df[df['Borough']!='Not assigned']

In [25]:
# assign Neighbourhood=Borough where Neighbourhood is 'Not assigned'
df[df['Neighbourhood']=='Not assigned']=df['Borough']

In [26]:
df.head()


,Postalcode,Borough,Neighbourhood


In [28]:
# group multiple Neighbourhood under one Postcode
temp_df=df.groupby('Postalcode')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighbourhood':'Neighbourhood_joined'},inplace=True)


In [29]:
# join the newly constructed joined data frame
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [30]:
# drop the Neighbourhood column
df_merge.drop(['Neighbourhood'],axis=1,inplace=True)

In [31]:
# drop duplicates from the data frame
df_merge.drop_duplicates(inplace=True)

,Borough,Postalcode,Neighbourhood_joined


In [32]:
# rename Neighbourhood_joined back to Neighbourhood
df_merge.rename(columns={'Neighbourhood_joined':'Neighbourhood'},inplace=True)

In [33]:
df_merge.head()


,Borough,Postalcode,Neighbourhood


In [34]:
df_merge.shape


(0, 3)

In [35]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [36]:
# takes a long time to get the data
#get_geocode('M3A')

# so we will use the csv sheet provided by Coursera as an alternative
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [37]:
geo_df.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [38]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, df_merge, on='Postalcode')

In [39]:
geo_merged.head()


,Postalcode,Latitude,Longitude,Borough,Neighbourhood
